In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import lxml
from urllib.request import Request, urlopen

In [3]:
WINE_REGION_URL = "https://winefolly.com/lifestyle/top-wine-regions-of-the-world/"
WINE_REGION_URL2 = "https://www.tonymappedit.com/top-10-wine-producing-countries/"
COUNTRIES = "https://lab.lmnixon.org/4th/worldcapitals.html"
COUNTRY_CODE_URL = "https://www.iban.com/country-codes"

In [4]:
'''scrape wine region data with coresponding wines'''
req = Request(WINE_REGION_URL2 , headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()

# html_text = requests.get(WINE_REGION_URL).text
soup = BeautifulSoup(webpage, "html.parser")

In [5]:
country1_df = []
countries = soup.find_all("ol")[1]
items = countries.find_all("li")

for country in items:
    country1_df.append(country.text.split(": "))

In [6]:
country2_df = []
countries2 = soup.find_all("ol")[2]
items = countries2.find_all("li")

for country in items:
    country2_df.append(country.text.split(": "))

In [7]:
country_df = country1_df + country2_df
country_df

[['Italy', '54.8'],
 ['France', '49.1'],
 ['Spain', '44.4'],
 ['United States', '23.9'],
 ['Argentina', '14.5'],
 ['Chile', '12.9'],
 ['Australia', '12.9'],
 ['Germany', '9.8'],
 ['South Africa', '9.5'],
 ['China', '9.3'],
 ['Russia', '6.5'],
 ['Portugal', '6.1'],
 ['Romania', '5.1'],
 ['Hungary', '3.6'],
 ['Brazil', '3.1'],
 ['New Zealand', '3.0'],
 ['Austria', '2.8'],
 ['Greece', '2.2'],
 ['Moldova', '1.9'],
 ['Switzerland', '1.1']]

In [8]:
country_production = pd.DataFrame(country_df).rename(columns = {0: "Country", 1: "WineProduction[HCL]"}).set_index("Country")
country_production

,WineProduction[HCL]
Country,
Italy,54.8
France,49.1
Spain,44.4
United States,23.9
Argentina,14.5
Chile,12.9
Australia,12.9
Germany,9.8
South Africa,9.5


In [9]:
dfs = pd.read_html(COUNTRIES)[0]


In [10]:
dfs.columns = dfs.iloc[0]

In [11]:
 countries_df = dfs.drop([201, 202, 203])

In [18]:
country_location = countries_df.set_index("Country").drop(columns = ["Latitude", "Longitude"])
country_location

,Capital
Country,
Country,Capital
Afghanistan,Kabul
Albania,Tirane
Algeria,Algiers
American Samoa,Pago Pago
...,...
Venezuela,Caracas
Viet Nam,Hanoi
Yugoslavia,Belgrade


In [19]:
code_dfs = pd.read_html(COUNTRY_CODE_URL)[0].drop(columns=['Alpha-2 code','Numeric']).set_index("Country")
code_dfs

,Alpha-3 code
Country,
Afghanistan,AFG
Åland Islands,ALA
Albania,ALB
Algeria,DZA
American Samoa,ASM
...,...
Wallis and Futuna,WLF
Western Sahara,ESH
Yemen,YEM


In [21]:
code_country = pd.merge(code_dfs, country_location, how="inner", on="Country").drop(columns= "Capital")
code_country

,Alpha-3 code
Country,
Afghanistan,AFG
Albania,ALB
Algeria,DZA
American Samoa,ASM
Andorra,AND
...,...
Uzbekistan,UZB
Vanuatu,VUT
Viet Nam,VNM


In [23]:
wine_production_HCL = pd.merge(country_production, code_country, how="right", on="Country").rename(columns = {'Alpha-3 code':'CODE'})
wine_production_HCL.to_dict()

{'WineProduction[HCL]': {'Italy': '54.8',
  'France': '49.1',
  'Spain': '44.4',
  'Argentina': '14.5',
  'Chile': '12.9',
  'Australia': '12.9',
  'Germany': '9.8',
  'South Africa': '9.5',
  'China': '9.3',
  'Portugal': '6.1',
  'Romania': '5.1',
  'Hungary': '3.6',
  'Brazil': '3.1',
  'New Zealand': '3.0',
  'Austria': '2.8',
  'Greece': '2.2',
  'Switzerland': '1.1',
  'Afghanistan': nan,
  'Albania': nan,
  'Algeria': nan,
  'American Samoa': nan,
  'Andorra': nan,
  'Angola': nan,
  'Antigua and Barbuda': nan,
  'Armenia': nan,
  'Aruba': nan,
  'Azerbaijan': nan,
  'Bahrain': nan,
  'Bangladesh': nan,
  'Barbados': nan,
  'Belarus': nan,
  'Belgium': nan,
  'Belize': nan,
  'Benin': nan,
  'Bhutan': nan,
  'Bosnia and Herzegovina': nan,
  'Botswana': nan,
  'Brunei Darussalam': nan,
  'Bulgaria': nan,
  'Burkina Faso': nan,
  'Burundi': nan,
  'Cambodia': nan,
  'Cameroon': nan,
  'Canada': nan,
  'Chad': nan,
  'Colombia': nan,
  'Costa Rica': nan,
  'Croatia': nan,
  'Cuba':

In [24]:
wine_production_HCL.to_csv("wine_production.csv")